In [1]:
!pip install transformers
!pip install chamd
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
import torch
import transformers as ppb
import warnings
from chamd import ChatReader
import random
from sklearn.decomposition import PCA
from sklearn.svm import SVC
warnings.filterwarnings('ignore')
import threading

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 37.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 43.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for chamd: filename=chamd-0.5.11-py3-none-any.whl size=15142 sha256=9688347ceda827560a3220f6b1318d8c03e6b6a7960db6fa898985df7edeec3c
  Stored in directory: /root/.cache/pip/wheels/c9/00/46/f2e02814d44b9e9956d7300e32245518d46d9db7bd09348485
Successfully built chamd


In [2]:

df = pd.read_csv('/content/drive/MyDrive/embeddings.csv')

new_embeddings = []
for embedding_str in df["embeddings"].to_list():
    embedding_str = embedding_str[1:-1]
    embedding_vals = []
    embeddings_strs = embedding_str.split(",")
    for embedding_strm in embeddings_strs:
        embedding_vals.append(float(embedding_strm))
    new_embeddings.append(embedding_vals)
df["embeddings"] = new_embeddings

In [3]:

#1. 50 runs of LOO for each architecture
runs = {}

def train_LR_and_save_preds(run, a):
  pred_list = []
  for i in range(0, 108):
      train_features, train_labels = df["embeddings"].tolist(), df["group"].tolist()
      test_embedding = train_features.pop(i)
      test_label = train_labels.pop(i)
      train_df = pd.DataFrame({"features": train_features, "labels": train_labels})
      valid_df = train_df.sample(n=11)
      train_df = train_df.drop(valid_df.index)
      train_features, train_labels = train_df["features"].tolist(), train_df["labels"].tolist()
      # Logistic Regression / Grid search
      parameters = {'C': np.linspace(0.0001, 100, 20)}
      grid_search = GridSearchCV(LogisticRegression(), parameters)
      grid_search.fit(train_features, train_labels)
      print("run " + str(run) + " i: " + str(i))
      lr_clf = LogisticRegression(C = grid_search.best_params_['C'])
      lr_clf.fit(train_features, train_labels)
      pred_list.append(lr_clf.predict([test_embedding])[0])
  pd.DataFrame({"pred" : pred_list}).to_csv("/content/drive/MyDrive/LR_preds_ada/" + str(run) + ".csv", index=True)


threads = []
for pool in range(0,25):
  t = threading.Thread(target = train_LR_and_save_preds, args = (pool, 7))
  threads.append(t)
  t.start()
  if pool >= 5:
    threads[0].join()
    threads.remove(threads[0])

result_df = {}
for N in [5,15,25,35,45,1]:
    scores = []
    for run in range(0,100):
        selected_runs = []
        for i_ensemble in range(0, N):
            idx = random.randint(0, 49)
            selected_runs.append(runs[idx])
        correct_votes = 0
        for df_idx in df.index:
            AD_votes = 0
            for selected_run in selected_runs:
                AD_votes = AD_votes + int(selected_run[df_idx])
            final_vote = 1 if AD_votes > (N - AD_votes) else 0
            correct_votes = (correct_votes + 1) if final_vote == [df["group"][df_idx]][0] else correct_votes
        scores.append(correct_votes / len(df.index))
    result_df[str(N)] = scores
#pd.DataFrame(result_df).to_csv("/content/drive/MyDrive/embeddings.csv", index=True)

run 2 i: 0
run 3 i: 0
run 5 i: 0
run 4 i: 0
run 1 i: 0
run 0 i: 0
run 3 i: 1
run 2 i: 1
run 5 i: 1
run 1 i: 1
run 4 i: 1
run 0 i: 1
run 2 i: 2
run 3 i: 2
run 4 i: 2
run 5 i: 2
run 0 i: 2
run 1 i: 2
run 2 i: 3
run 3 i: 3
run 4 i: 3
run 0 i: 3
run 1 i: 3
run 5 i: 3
run 2 i: 4
run 3 i: 4
run 4 i: 4
run 0 i: 4
run 1 i: 4
run 5 i: 4
run 3 i: 5
run 2 i: 5
run 4 i: 5
run 0 i: 5
run 1 i: 5
run 5 i: 5
run 3 i: 6
run 2 i: 6
run 4 i: 6
run 1 i: 6
run 0 i: 6
run 5 i: 6
run 3 i: 7
run 2 i: 7
run 4 i: 7
run 1 i: 7
run 0 i: 7
run 5 i: 7
run 3 i: 8
run 4 i: 8
run 2 i: 8
run 0 i: 8
run 1 i: 8
run 5 i: 8
run 3 i: 9
run 4 i: 9
run 2 i: 9
run 0 i: 9
run 1 i: 9
run 5 i: 9
run 3 i: 10
run 4 i: 10
run 2 i: 10
run 0 i: 10
run 1 i: 10
run 5 i: 10
run 3 i: 11
run 4 i: 11
run 2 i: 11
run 0 i: 11
run 1 i: 11
run 5 i: 11
run 3 i: 12
run 4 i: 12
run 2 i: 12
run 1 i: 12
run 0 i: 12
run 5 i: 12
run 3 i: 13
run 4 i: 13
run 2 i: 13
run 1 i: 13
run 0 i: 13
run 5 i: 13
run 3 i: 14
run 2 i: 14
run 4 i: 14
run 0 i: 14
run 

KeyError: ignored